In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow import keras

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             #mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             #mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             #) 
    # Draw pose connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
    #                         mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
    #                         mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                         ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    #pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['i love you','hello', 'I', 'to', 'forgive', 'hill', 'meet', 'want', 'yes', 'go', 'drink', 'now'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [88]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [89]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'i love you': 0,
 'hello': 1,
 'I': 2,
 'to': 3,
 'forgive': 4,
 'hill': 5,
 'meet': 6,
 'want': 7,
 'yes': 8,
 'go': 9,
 'drink': 10,
 'now': 11}

In [90]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [91]:
X = np.array(sequences)

In [92]:
y = to_categorical(labels).astype(int)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [94]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, SimpleRNN, GRU
from tensorflow.keras.callbacks import TensorBoard

In [95]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [136]:
model = Sequential()
model.add((SimpleRNN(64, return_sequences=True, activation='relu', input_shape=(30,126))))
model.add((SimpleRNN(128, return_sequences=True, activation='relu')))
model.add((SimpleRNN(64, return_sequences=False, activation='relu')))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [137]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [138]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
10/10 [==============================] - 2s 16ms/step - loss: 2.4316 - categorical_accuracy: 0.1209
Epoch 2/200
10/10 [==============================] - 0s 19ms/step - loss: 1.9995 - categorical_accuracy: 0.2778
Epoch 3/200
10/10 [==============================] - 0s 17ms/step - loss: 1.7495 - categorical_accuracy: 0.4510
Epoch 4/200
10/10 [==============================] - 0s 20ms/step - loss: 1.5117 - categorical_accuracy: 0.5458
Epoch 5/200
10/10 [==============================] - 0s 18ms/step - loss: 1.3648 - categorical_accuracy: 0.5490
Epoch 6/200
10/10 [==============================] - 0s 26ms/step - loss: 1.2435 - categorical_accuracy: 0.5490
Epoch 7/200
10/10 [==============================] - 0s 27ms/step - loss: 1.0814 - categorical_accuracy: 0.6078
Epoch 8/200
10/10 [==============================] - 0s 21ms/step - loss: 0.9622 - categorical_accuracy: 0.6863
Epoch 9/200
10/10 [==============================] - 0s 28ms/step - loss: 0.9440 - categorical_accuracy:

10/10 [==============================] - 0s 15ms/step - loss: 0.0032 - categorical_accuracy: 1.0000
Epoch 146/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0032 - categorical_accuracy: 1.0000
Epoch 147/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0031 - categorical_accuracy: 1.0000
Epoch 148/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0031 - categorical_accuracy: 1.0000
Epoch 149/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0031 - categorical_accuracy: 1.0000
Epoch 150/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0030 - categorical_accuracy: 1.0000
Epoch 151/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0030 - categorical_accuracy: 1.0000
Epoch 152/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0030 - categorical_accuracy: 1.0000
Epoch 153/200
10/10 [==============================] - 0s 14ms/step - loss: 0.0029 - categorical_accur

In [139]:
res = model.predict(X_test)

In [140]:
res

array([[2.81403300e-06, 2.64602655e-04, 1.09090011e-04, 1.07844782e-04,
        2.77123268e-04, 9.98770058e-01, 8.12883663e-05, 5.59648252e-05,
        2.14824802e-04, 1.69544364e-05, 7.23141275e-05, 2.71223507e-05],
       [5.57163912e-05, 2.62243178e-04, 5.41218673e-04, 1.37080147e-04,
        3.30387570e-06, 1.69213614e-04, 1.14650975e-05, 6.33404765e-04,
        9.97751296e-01, 1.56826442e-04, 2.51599355e-04, 2.66583211e-05],
       [1.98530179e-05, 3.57442059e-06, 7.71672712e-05, 4.03814658e-04,
        2.30063288e-05, 6.98890217e-05, 9.98234987e-01, 3.03710054e-04,
        1.41870678e-05, 1.76916816e-04, 1.62712004e-05, 6.56648364e-04],
       [3.20677755e-05, 1.04197170e-04, 4.63645847e-04, 1.42932666e-04,
        9.97951806e-01, 4.33794019e-04, 4.04067723e-05, 2.29474725e-04,
        4.03170679e-06, 2.23919007e-04, 1.13561396e-06, 3.72468203e-04],
       [2.93898716e-04, 4.31801453e-07, 3.52324605e-05, 5.50836012e-05,
        5.04294934e-04, 6.67873901e-05, 4.07267798e-04, 5.02

In [141]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [142]:
yhat = model.predict(X_test)

In [143]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [144]:
multilabel_confusion_matrix(ytrue, yhat) 

array([[[46,  0],
        [ 0,  8]],

       [[51,  0],
        [ 0,  3]],

       [[47,  2],
        [ 4,  1]],

       [[52,  0],
        [ 0,  2]],

       [[45,  0],
        [ 0,  9]],

       [[51,  0],
        [ 0,  3]],

       [[50,  0],
        [ 0,  4]],

       [[51,  0],
        [ 0,  3]],

       [[45,  3],
        [ 2,  4]],

       [[51,  0],
        [ 0,  3]],

       [[49,  1],
        [ 0,  4]],

       [[50,  0],
        [ 0,  4]]], dtype=int64)

In [145]:
accuracy_score(ytrue, yhat)

0.8888888888888888

In [8]:
model = keras.models.load_model('final_v1.h5')

In [9]:
colors = [(245,117,16), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245),(117,245,16), 
          (16,117,245),( 245,117,16), (117,245,16), (16,117,245), ( 245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [21]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.85

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        cv2.waitKey(50)
        sequence = sequence[-30:]
        cv2.waitKey(50)
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilitiesqqqqqqqq
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
to
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_bas

drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.sol

forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
forgive
<class 'mediapipe

In [155]:
model.save('final_v1.h5') #['i love you','hello', 'I', 'to', 'forgive', 'hill', 'meet', 'want', 'yes', 'go', 'drink'
#, 'now']

In [11]:
!pip install tensorflowjs

In [12]:
import tensorflowjs as tfjs

In [15]:
tfjs.converters.save_keras_model(model, 'models')